In [56]:
import requests
import time
import re
from bs4 import BeautifulSoup
import csv


In [155]:

PTT_URL = 'https://www.ptt.cc'
API_KEY = '26266720e9aa08a4477f6ed04c1404ec'


def get_web_page(url):
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text


def get_articles(dom, date):
    soup = BeautifulSoup(dom, 'html5lib')

    # 取得上一頁的連結
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']

    articles = []  # 儲存取得的文章資料
    authortotal=[]
    divs = soup.find_all('div', 'r-ent')
    for d in divs:
        if d.find('div', 'date').text.strip() == date:  # 發文日期正確
            # 取得推文數
            push_count = 0
            push_str = d.find('div', 'nrec').text
            if push_str:
                try:
                    push_count = int(push_str)  # 轉換字串為數字
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2', ...
                    # 若不是, 不做任何事，push_count 保持為 0
                    if push_str == '爆':
                        push_count = 99
                    elif push_str.startswith('X'):
                        push_count = -10

            # 取得文章連結及標題
            if d.find('a'):  # 有超連結，表示文章存在，未被刪除
                href = d.find('a')['href']
                title = d.find('a').text
                author = d.find('div', 'author').text if d.find('div', 'author') else ''
                articles.append({
                    'title': title,
                    'href': href,
                    'push_count': push_count,
                    'author': author
                })
                authortotal.append(author)
    return articles, prev_url,authortotal


def get_ip(dom):
    # e.g., ※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 27.52.6.175
    pattern = '來自: \d+\.\d+\.\d+\.\d+'
    match = re.search(pattern, dom)
    if match:
        return match.group(0).replace('來自: ', '')
    else:
        return None


def get_country(ip):
    if ip:
        url = 'http://api.ipstack.com/{}?access_key={}'.format(ip, API_KEY)
        data = requests.get(url).json()
        country_name = data['country_name'] if data['country_name'] else None
        return country_name
    return None


if __name__ == '__main__':
    print('取得今日文章列表...')
    current_page = get_web_page(PTT_URL + '/bbs/Gossiping/index.html')
    if current_page:
        articles = []  # 全部的今日文章
        author=[]
        countryT=[]
        title=[]
        today = time.strftime('%m/%d').lstrip('0')  # 今天日期, 去掉開頭的 '0' 以符合 PTT 網站格式
        current_articles, prev_url,authortotal = get_articles(current_page, today)  # 目前頁面的今日文章
        while current_articles:  # 若目前頁面有今日文章則加入 articles，並回到上一頁繼續尋找是否有今日文章
            articles += current_articles
            current_page = get_web_page(PTT_URL + prev_url)
            current_articles, prev_url,authortotal = get_articles(current_page, today)
        print('共 %d 篇文章' % (len(articles)))

        # 已取得文章列表，開始進入各文章尋找發文者 IP
        print('取得前 100 篇文章 IP')
        country_to_count = dict()
        for article in articles[:100]:
            print('查詢 IP:', article['title'])
            page = get_web_page(PTT_URL + article['href'])
            if page:
                ip = get_ip(page)
                country = get_country(ip)
                if country in country_to_count.keys():
                    country_to_count[country] += 1
                else:
                    country_to_count[country] = 1
            print("來自",country, end='')
            print("   ","作者是",article['author'])
            author.append(article['author'])
            title.append(article['title'])
            countryT.append(country)
        # 印出各國 IP 次數資訊
        print('各國 IP 分布')
        for k, v in country_to_count.items():
            print(k, v)

取得今日文章列表...
共 1018 篇文章
取得前 100 篇文章 IP
查詢 IP: [新聞] 明文規定陳抗禁止設攤 6/1韓國瑜凱道造勢
來自 Taiwan    作者是 teramars
查詢 IP: Re: [問卦] 全世界有哪些政黨是公開希望被敵國統治
來自 Taiwan    作者是 AGIknight
查詢 IP: [問卦] 男躲國小廁所！偷拍5女童脫褲子如廁
來自 Germany    作者是 norvasc
查詢 IP: [新聞] 東莞數百公交司機汽車站維權 討工齡賠償
來自 United States    作者是 eeth
查詢 IP: Re: [新聞] 打死3歲兒 死了扔山谷 狠父遭判9年半
來自 Taiwan    作者是 cirin
查詢 IP: [新聞] 駕駛突開車門！男未戴安全帽 遭後方機車
來自 Taiwan    作者是 hw102050
查詢 IP: [臉書] Emmy Hu
來自 Taiwan    作者是 cycling
查詢 IP: Re: [爆卦] 扁無罪卻被紅杉軍惡搞 扁:因為施明德恨我
來自 Taiwan    作者是 EvilPrada
查詢 IP: [問卦] 沒繳稅會怎樣?
來自 Taiwan    作者是 ordonez
查詢 IP: Re: [問卦] 毛澤東抓到竊聽器的時候怎麼不生氣
來自 Taiwan    作者是 NTUSTNTU
各國 IP 分布
Taiwan 8
Germany 1
United States 1


In [156]:
with open('output.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=';') 
            writer.writerow(['文章標題', '作者', '國家'])
            for i in range(len(author)):
                writer.writerow([title[i],author[i],countryT[i]])
